In [ ]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

%matplotlib widget

In [ ]:
# Import the relevant modules
import sousvide.synthesize.rollout_generator as rg
import sousvide.synthesize.observation_generator as og
import sousvide.instruct.train_policy as tp
import sousvide.visualize.plot_synthesize as ps
import sousvide.visualize.plot_learning as pl
import sousvide.flight.deploy_figs as df

In [ ]:
cohort = "transformers"             # cohort name

data_method = "eval_single"          # method name

scene = "mid_gate"                  # scene name

courses = [                         # course names
    "hover",
    ]   

roster = [
    "hsDavion",                     # sifu_testbed
    ]

In [ ]:
# Generate Rollouts
rg.generate_rollout_data(cohort,courses,scene,data_method)

# Review the Rollout Data
ps.plot_rollout_data(cohort)

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import os
import torch
import figs.visualize.generate_videos as gv

K = np.array([
    [ 462.956,   0.000, 323.076],
    [   0.000, 463.002, 181.184],
    [   0.000,   0.000,   1.000]
])
Tc2i = np.array([
    [ 0.00000,  0.00000, -1.00000,  0.10000],
    [ 1.00000,  0.00000,  0.00000, -0.03000],
    [ 0.00000, -1.00000,  0.00000, -0.01000],
    [ 0.00000,  0.00000,  0.00000,  1.00000]
])
Ti2c = np.linalg.inv(Tc2i)
Rgl2cv = np.array([
    [ 1.0, 0.0, 0.0],
    [ 0.0,-1.0, 0.0],
    [ 0.0, 0.0,-1.0]
])

def pose2proj(xcr,xds):
    # Goal Frame to World Frame
    Tds2wd = np.eye(4)
    Tds2wd[0:3,0:3] = R.from_quat(xds[6:10]).as_matrix()
    Tds2wd[0:3,3] = xds[0:3]

    # World Frame to Current Frame
    Twd2cr = np.eye(4)
    Twd2cr[0:3,0:3] = R.from_quat(xcr[6:10]).as_matrix().T
    Twd2cr[0:3,3] = -Twd2cr[0:3,0:3]@xcr[0:3]

    # Position in camera frame
    rc = Ti2c@Twd2cr@Tds2wd@rg
    rc = rc[0:3,:]
    # Switch to OpenCV convention
    rc = Rgl2cv@rc
    # Project to Image Plane
    r_pr = K@rc

In [ ]:
# Load Trajectories
course_directory = os.path.join("..","cohorts",cohort,"rollout_data","robustness_track")
# course_directory = os.path.join("..","cohorts",cohort,"rollout_data","line")
# course_directory = os.path.join("..","cohorts",cohort,"rollout_data","hover")

Trajectories = torch.load(os.path.join(course_directory,"trajectories","trajectories000.pt"))
Images = torch.load(os.path.join(course_directory,"images","images000.pt"))

trajectories = Trajectories[0]
images = Images[0]

Tro,Xro,tXd = trajectories["Tro"],trajectories["Xro"],trajectories["tXd"]


Iro = 1*images["images"]
Nro = len(Tro)

In [ ]:
dst,nst = 10,4
Nst = dst*nst
for i in range(Nro):
    idx_rf = np.arange(i+dst,i+Nst+1,dst)
    idx_rf = np.clip(idx_rf,0,Nro-1)
    Xrf = Xro[:,idx_rf]


In [ ]:
nft,nb = 20,2
Nres,Nro = 100,

# Extract future poses
ob = np.zeros((4,1))
xb = np.zeros((4,Nres))
yb = np.zeros((4,Nres))
zb = np.zeros((4,Nres))
xb[0,:] = np.linspace(0.0,0.3,Nres)
yb[1,:] = np.linspace(0.0,0.3,Nres)
zb[2,:] = np.linspace(0.0,0.1,Nres)
xb[3,:],yb[3,:],zb[3,:] = 1.0,1.0,1.0

rinit = [xb,yb,zb]
for i in range(Nro):
    # Get index
    g = np.min([i+nft,len(Tro)-1])

    # Goal Frame to World Frame
    Tg2w = np.eye(4)
    Tg2w[0:3,0:3] = R.from_quat(Xro[6:10,g]).as_matrix()
    Tg2w[0:3,3] = Xro[0:3,g]

    # World Frame to Current Frame
    Tw2i = np.eye(4)
    Tw2i[0:3,0:3] = R.from_quat(Xro[6:10,i]).as_matrix().T
    Tw2i[0:3,3] = -Tw2i[0:3,0:3]@Xro[0:3,i]

    refs = []
    for idx,rg in enumerate(rinit):
        # Position in camera frame
        rc = Ti2c@Tw2i@Tg2w@rg
        rc = rc[0:3,:]

        # Switch to OpenCV convention
        rc = Rgl2cv@rc

        # Project to Image Plane
        r_pr = K@rc
        # Append to list
        refs.append(r_pr)

    # Handle Reference Arrows
    for j in range(Nres):
        for k in range(3):
            z = refs[k][2,j]
            width  = int(refs[k][0,j]/z)
            height = int(refs[k][1,j]/z)

            if z > 0 and width > 0 and width < 640 and height > 0 and height < 360:
                Iro[i,height-nb:height+nb,width-nb:width+nb,:] = 0
                Iro[i,height-nb:height+nb,width-nb:width+nb,k] = 255

gv.images_to_mp4(Iro,'output.mp4', 20)
